In [41]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import os

In [42]:
df = pd.read_csv("label.csv")

In [ ]:
try:
    df["label"] = df["label"].apply(json.loads)  
except (json.JSONDecodeError, TypeError):
    print("Error parsing the `label` column. Ensure the data is valid JSON.")

new_df = pd.DataFrame([item for sublist in df["label"] for item in sublist])

new_df["timeserieslabel"] = new_df["timeserieslabels"].apply(lambda x: x[0])

new_df = new_df.drop(columns=["timeserieslabels"])

new_df

,start,end,instant,timeserieslabel
0,0.032967,1.468423,False,fist
1,1.486967,2.322828,False,rest
2,2.340685,4.266531,False,fist
3,4.284388,4.920384,False,rest
4,4.938929,6.628508,False,fist
...,...,...,...,...
83,44.003236,48.018391,False,rest
84,48.018391,51.979286,False,rest
85,51.971044,56.022600,False,rest
86,56.004056,59.964952,False,rest


In [ ]:
gest_arr = []  

for _, row in new_df.iterrows():
    label_file = f".\\time_series\\{row['timeserieslabel']}.csv"
    
    if os.path.exists(label_file):
        gest_df = pd.read_csv(label_file)
        
        amp = gest_df[(gest_df["time"] >= row["start"]) & (gest_df["time"] <= row["end"])].copy()

        amp['label'] = row["timeserieslabel"]
        
        gest_arr.append(amp)
    else:
        print(f"File not found for label: {row['timeserieslabel']}")

final_df = pd.concat(gest_arr, ignore_index=True)

final_df

,time,amplitude,label
0,0.033654,1255,fist
1,0.034341,1258,fist
2,0.035028,1260,fist
3,0.035715,1262,fist
4,0.036402,1260,fist
...,...,...,...
268105,61.760988,1290,rest
268106,61.761675,1289,rest
268107,61.762362,1290,rest
268108,61.763049,1287,rest


In [ ]:
prev_time = None
increment_time = 0
offset = 0.0005

for idx, row in final_df.iterrows():

    if row["time"] == 0.0:
        increment_time = previous_time
        final_df.at[idx, "time"] = increment_time + offset
    
    else:
        final_df.at[idx, "time"] += increment_time
    
    previous_time = final_df.at[idx, "time"]
    
final_df


,time,amplitude,label
0,0.033654,1255,fist
1,0.034341,1258,fist
2,0.035028,1260,fist
3,0.035715,1262,fist
4,0.036402,1260,fist
...,...,...,...
268105,185.324175,1290,rest
268106,185.324861,1289,rest
268107,185.325548,1290,rest
268108,185.326235,1287,rest


In [46]:
final_df.to_csv("dataset2.csv", index=False)

In [ ]:
df = pd.read_csv("dataset2.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268110 entries, 0 to 268109
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   time       268110 non-null  float64
 1   amplitude  268110 non-null  int64  
 2   label      268110 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 6.1+ MB
